In [1]:
# app.py (Flask Backend)

from flask import Flask, request, jsonify
from flask_cors import CORS
import tensorflow_datasets as tfds
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
from nltk.corpus import stopwords
import numpy as np
import tensorflow as tf
from PIL import Image
import io
import base64
import numpy as np

app = Flask(__name__)
CORS(app)

# Initialize NLTK and load stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load the IMDB dataset and model (same preprocessing steps as in the original code)
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)
train_data, test_data = dataset['train'], dataset['test']

def preprocess_text(text):
    if isinstance(text, tf.Tensor):
        text_str = text.numpy().decode('utf-8')
    else:
        text_str = text
    words = text_str.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)

# Preprocess train data
train_texts = [preprocess_text(text) for text, label in train_data]
train_labels = [label.numpy() for text, label in train_data]

# Train the TF-IDF vectorizer and logistic regression model
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(train_texts)
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, train_labels)

# Sentiment prediction function
def predict_sentiment(text):
    text_processed = preprocess_text(text)
    text_tfidf = vectorizer.transform([text_processed])
    prediction = model.predict(text_tfidf)
    return "Positive" if prediction[0] == 1 else "Negative"

# API endpoint for sentiment analysis
@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    text = data.get('text', None)
    if text:
        sentiment = predict_sentiment(text)
        return jsonify({"sentiment": sentiment})
    else:
        return jsonify({"error": "No text provided"}), 400

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.34:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\user\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
